<a href="https://colab.research.google.com/github/sagarikakaramadi/Information_Security/blob/main/Information_security_ECC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cryptography


In [ ]:

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

#  ECC private-public key pair
def generate_keys():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

# derive shared secret
def derive_shared_key(private_key, recipient_public_key):
    shared_key = private_key.exchange(ec.ECDH(), recipient_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
        backend=default_backend()
    ).derive(shared_key)
    return derived_key

def encrypt_message(derived_key, plaintext):
    iv = os.urandom(12)
    aesgcm = Cipher(algorithms.AES(derived_key), modes.GCM(iv), backend=default_backend()).encryptor()
    ciphertext = aesgcm.update(plaintext) + aesgcm.finalize()
    return iv, ciphertext, aesgcm.tag

def decrypt_message(derived_key, iv, ciphertext, tag):
    aesgcm = Cipher(algorithms.AES(derived_key), modes.GCM(iv, tag), backend=default_backend()).decryptor()
    plaintext = aesgcm.update(ciphertext) + aesgcm.finalize()
    return plaintext

if __name__ == "__main__":
    sender_private_key, sender_public_key = generate_keys()
    recipient_private_key, recipient_public_key = generate_keys()

    recipient_public_key_bytes = recipient_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    print(f"Recipient Public Key: {recipient_public_key_bytes.decode()}")

    shared_secret_sender = derive_shared_key(sender_private_key, recipient_public_key)
    shared_secret_recipient = derive_shared_key(recipient_private_key, sender_public_key)

    print(f"Sender's Derived Key: {shared_secret_sender}")
    print(f"Recipient's Derived Key: {shared_secret_recipient}")

    assert shared_secret_sender == shared_secret_recipient, "Shared secrets do not match!"

    # to be encrypted
    message = b"Secret message for ECC encryption!"

    # Sender encrypts
    iv, ciphertext, tag = encrypt_message(shared_secret_sender, message)
    print(f"Ciphertext: {ciphertext}")
 # Verify
    assert message == decrypted_message, "Decryption failed, messages do not match!"

    print("Encryption and decryption successful, messages match.")
    # Recipient decrypts
    decrypted_message = decrypt_message(shared_secret_recipient, iv, ciphertext, tag)
    print(f"Decrypted Message: {decrypted_message.decode()}")






Recipient Public Key: -----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAED2SmI+j/pS6Jc2SjwLmPUTJulU6k
q53Wbp5RS83dpQso2vsnVxuk8OHqbb29wvmxpgdOrgoC9AGmfaip2NaXLA==
-----END PUBLIC KEY-----

Sender's Derived Key: b'x%\x9e\xbcJ8\xc0\rD\x03\x87~\xf0\x92=[k\x1dH\x8f\x18\x80\x01\xc4s\x83U\xdf*\xf9\xe2\xb7'
Recipient's Derived Key: b'x%\x9e\xbcJ8\xc0\rD\x03\x87~\xf0\x92=[k\x1dH\x8f\x18\x80\x01\xc4s\x83U\xdf*\xf9\xe2\xb7'
Ciphertext: b'\x16\xb4in\xb7iRo\x9ao(\x06\xf6B\x81\x0f\x9d\x84\xe83\x12\xe5\x85{]\xb38\xe1\x89\xad]\xd0\x93B'
Encryption and decryption successful, messages match.
Decrypted Message: Secret message for ECC encryption!
